## Setup and Import Libraries

In [2]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from typing import List

In [3]:
load_dotenv()

True

In [4]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

In [5]:
llm = ChatGroq(model="Gemma2-9b-It")

## Documents

LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

- page_content: a string representing the content;
- metadata: a dict containing arbitrary metadata.

The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

In [6]:
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [7]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

## Embeddings and Vector Store

In [9]:
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

In [10]:
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embeddings
)

## Similarity Search

In [11]:
vector_store.similarity_search("Cat")

[Document(id='35321cce-c7ec-4ff9-8934-72b11e0c47e6', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='b7a2cc7b-2d17-47cd-b03a-ebddd4a491a0', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='6f98df52-5d8e-4c06-a67a-fe9b504d3cc4', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='8cb28968-86d9-4684-9c20-bb2f1fb709b5', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

## Async Query

In [12]:
await vector_store.asimilarity_search("Cat")

[Document(id='35321cce-c7ec-4ff9-8934-72b11e0c47e6', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='b7a2cc7b-2d17-47cd-b03a-ebddd4a491a0', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='6f98df52-5d8e-4c06-a67a-fe9b504d3cc4', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='8cb28968-86d9-4684-9c20-bb2f1fb709b5', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

## Similarity Search with Score

In [13]:
vector_store.similarity_search_with_score("Cat")

[(Document(id='35321cce-c7ec-4ff9-8934-72b11e0c47e6', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.935105562210083),
 (Document(id='b7a2cc7b-2d17-47cd-b03a-ebddd4a491a0', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.574089765548706),
 (Document(id='6f98df52-5d8e-4c06-a67a-fe9b504d3cc4', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.5956902503967285),
 (Document(id='8cb28968-86d9-4684-9c20-bb2f1fb709b5', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.6657922267913818)]

## Retrievers

### Runnable Lambda

In [14]:
retriever = RunnableLambda(vector_store.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(id='35321cce-c7ec-4ff9-8934-72b11e0c47e6', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='b7a2cc7b-2d17-47cd-b03a-ebddd4a491a0', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

### Retriever Method

In [15]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)

retriever.batch(["cat","dog"])

[[Document(id='35321cce-c7ec-4ff9-8934-72b11e0c47e6', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='b7a2cc7b-2d17-47cd-b03a-ebddd4a491a0', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

## Retriever with Chain

In [16]:
message = """ Answer this question using the provided context only.

{question}

Context:
{context}
"""

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", message)
    ]
)

In [18]:
rag_chain = {
    "context": retriever,
    "question": RunnablePassthrough()
} | prompt | llm

In [19]:
response = rag_chain.invoke("Tell me about dogs")
print(response.content)

Dogs are great companions, known for their loyalty and friendliness.  

